In [1]:
import pandas as pd
import numpy as np
import warnings, pickle, os, json, sqlalchemy
warnings.filterwarnings("ignore")

gap = 1
count = 20
aluminum_division = 12
copper_division = 72
copper_limit = 300
aluminum_limit = 60
max_k = 10

In [2]:
input_ = {
    "work_id": "FC236625",
    "op": 2,
    "model_id": "20230807120000000",
    "speed": "590",
    "l_angle_ori": "127",
    "l_weight_ori": "0.85",
    "f_angle_ori": "155",
    "f_weight_ori": "2.10",
    "material": "copper", # 'aluminum', 'copper'
}

In [4]:
work_id = input_["work_id"]
op = input_["op"]
model_id = input_["model_id"]
speed = int(input_["speed"])
l_angle_ori  = int(input_["l_angle_ori"])
l_weight_ori = round(float(input_["l_weight_ori"]))
f_angle_ori  = int(input_["f_angle_ori"])
f_weight_ori = round(float(input_["f_weight_ori"]))
material = input_["material"]
work_id, op, speed, l_angle_ori, l_weight_ori, f_angle_ori, f_weight_ori, material

('FC239512', '1', 1000, 315, 107, 338, 54, 'aluminum')

In [5]:
root = r"C:\Users\tzuli\Documents\python\teco"

# 取得predict位置
pred_path = os.path.join(root, "data", "predict")        
os.makedirs(pred_path, exist_ok = True)
output_json = os.path.join(pred_path, "output.json")

# 取得model位置     
model_detail = os.path.join(root, "data", "train", model_id, "model")

config_path = os.path.join(root, "prog", "config.json")
with open(config_path) as f:
    config = json.load(f)

### Load model

In [6]:
side = "L"
features = pickle.load(open(os.path.join(model_detail, f"feat_order.pkl"), "rb"))
l_outlier_boundary = pickle.load(open(os.path.join(model_detail, f"{side}_outlier_boundary.pkl"), "rb"))
l_skew_feat = pickle.load(open(os.path.join(model_detail, f"{side}_skew_feat.pkl"), "rb"))
l_pt = pickle.load(open(os.path.join(model_detail, f"{side}_power_tf.pkl"), "rb"))
l_scaler = pickle.load(open(os.path.join(model_detail, f"{side}_scaler.pkl"), "rb"))
l_model = pickle.load(open(os.path.join(model_detail, f"{side}_model.pkl"), "rb"))

side = "F"
f_outlier_boundary = pickle.load(open(os.path.join(model_detail, f"{side}_outlier_boundary.pkl"), "rb"))
f_skew_feat = pickle.load(open(os.path.join(model_detail, f"{side}_skew_feat.pkl"), "rb"))
f_pt = pickle.load(open(os.path.join(model_detail, f"{side}_power_tf.pkl"), "rb"))
f_scaler = pickle.load(open(os.path.join(model_detail, f"{side}_scaler.pkl"), "rb"))
f_model = pickle.load(open(os.path.join(model_detail, f"{side}_model.pkl"), "rb"))

### Get data from DB

In [7]:
con_info = f'mysql+pymysql://{config["user"]}:{config["password"]}@{config["host"]}:{config["port"]}/{config["database"]}'
conn = sqlalchemy.create_engine(con_info)

query = f'SELECT * FROM {config["table"]} WHERE (work_id = "{work_id}")'
df_db = pd.read_sql(query, conn).sort_values("op")
df_db

,work_id,op,model_id,material,speed,l_angle_ori,l_weight_ori,f_angle_ori,f_weight_ori,l_angle_ai,...,f_angle_ai,f_weight_ai,l_angle_act,l_weight_act,f_angle_act,f_weight_act,create_time,create_by,modify_time,modify_by
0,FC239512,1,20230902202809180,aluminum,1000,315,107.0,338,54.0,330,...,330,40,None,None,None,None,2023-09-08 12:42:44,admin,2023-09-08 12:42:44,admin


In [8]:
# 可填補角度要與起始角度同範圍，候選角度以初始角度為矛點
if not df_db.empty:
    l_angle_ori, f_angle_ori = df_db.loc[0, "l_angle_ori"], df_db.loc[0, "f_angle_ori"]
l_angle_ori, f_angle_ori

(315, 338)

### Generate candidate values

In [9]:
def generate_weight(weight_ori, gap, count, weight_limit):
    end = round((count + 1) * gap) # 候選重量的結束範圍
    weight_ori = min(weight_ori, weight_limit) # 重量不可大於上限
    
    # 產生候選重量
    weight_change = []
    for i in range(0, end, gap):
        weight_change.extend([max(0, weight_ori + i), max(0, weight_ori - i)])

    # 刪除相同的值
    weight_change = sorted(list(set(weight_change)))
    weight_change = np.array(weight_change)

    # 重量不可大於上限
    weight_change = weight_change[weight_change <= weight_limit]

    return weight_change

In [10]:
weight_limit = aluminum_limit if material == "aluminum" else copper_limit

l_weight_change = generate_weight(l_weight_ori, gap, count, weight_limit)
f_weight_change = generate_weight(f_weight_ori, gap, count, weight_limit)

print(f"l_weight_change = {l_weight_change}")
print(f"f_weight_change = {f_weight_change}")

l_weight_change = [40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60]
f_weight_change = [34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57
 58 59 60]


In [11]:
def generate_angle(angle_ori, material, aluminum_division, copper_division, max_k):
    # 產生候選角度
    if material == "aluminum":
        angle_init = np.linspace(0, 360, (aluminum_division + 1))
        k = min(round(90 / (360 / aluminum_division)), max_k) # 候選組合只可在同象限內
    else:    
        angle_init = np.linspace(0, 360, (copper_division + 1))
        k = min(round(90 / (360 / copper_division)), max_k) # 候選組合只可在同象限內

    closest_index = np.argsort(np.abs(angle_init - angle_ori))[:k] # 與angle_ori最接近的K個角度
    angle_change0 = angle_init[closest_index]


    # 調整候選較度
    angle_change = []
    for angle in angle_change0:
        if angle > 360:
            angle_change.append(angle - 360)
        elif angle < 0:
            angle_change.append(360 + angle)
        else:
            angle_change.append(angle)
    
    # 排序候選角度
    angle_change = sorted(angle_change)

    return angle_change, angle_init

In [12]:
l_angle_change, l_angle_init = generate_angle(l_angle_ori, material, aluminum_division, copper_division, max_k)
f_angle_change, f_angle_init = generate_angle(f_angle_ori, material, aluminum_division, copper_division, max_k)
    
print(f"l_angle_change = {l_angle_change}")
print(f"f_angle_change = {f_angle_change}")

l_angle_change = [270.0, 300.0, 330.0]
f_angle_change = [300.0, 330.0, 360.0]


In [13]:
# 產生對向候選重量
l_weight_change_opposite = l_weight_change * -1
f_weight_change_opposite = f_weight_change * -1

print(f"l_weight_change_opposite = {l_weight_change_opposite}")
print(f"f_weight_change_opposite = {f_weight_change_opposite}")

l_weight_change_opposite = [-40 -41 -42 -43 -44 -45 -46 -47 -48 -49 -50 -51 -52 -53 -54 -55 -56 -57
 -58 -59 -60]
f_weight_change_opposite = [-34 -35 -36 -37 -38 -39 -40 -41 -42 -43 -44 -45 -46 -47 -48 -49 -50 -51
 -52 -53 -54 -55 -56 -57 -58 -59 -60]


In [14]:
# 產生對向候選角度
l_angle_change_opposite = [(angle - 180) if angle > 180 else (angle + 180) for angle in l_angle_change]
f_angle_change_opposite = [(angle - 180) if angle > 180 else (angle + 180) for angle in f_angle_change]

print(f"l_angle_change_opposite = {l_angle_change_opposite}")
print(f"f_angle_change_opposite = {f_angle_change_opposite}")

l_angle_change_opposite = [90.0, 120.0, 150.0]
f_angle_change_opposite = [120.0, 150.0, 180.0]


### Generate candidate combinations

In [15]:
# 循環生成所有组合
index = 0
all_combinations = []
for a in l_angle_change :
    for b in l_weight_change:
        for c in f_angle_change:
            for d in f_weight_change:
                x = [speed, l_angle_ori, l_weight_ori, f_angle_ori, f_weight_ori, a, b, c, d]
                all_combinations.append(x)

for a in l_angle_change_opposite :
    for b in l_weight_change_opposite:
        for c in f_angle_change_opposite:
            for d in f_weight_change_opposite:
                x = [speed, l_angle_ori, l_weight_ori, f_angle_ori, f_weight_ori, a, b, c, d]
                all_combinations.append(x)
                
len(all_combinations)

10206

In [16]:
df_comb = pd.DataFrame(
    all_combinations, 
    columns = ['平衡轉速', '初始_L側角度', '初始_L側不平衡量', '初始_F側角度', '初始_F側不平衡量', '平衡_L側角度', '平衡_L側配重', '平衡_F側角度', '平衡_F側配重']
    )
print(df_comb.shape)
df_comb.head(3)

(10206, 9)


,平衡轉速,初始_L側角度,初始_L側不平衡量,初始_F側角度,初始_F側不平衡量,平衡_L側角度,平衡_L側配重,平衡_F側角度,平衡_F側配重
0,1000,315,107,338,54,270.0,40,300.0,34
1,1000,315,107,338,54,270.0,40,300.0,35
2,1000,315,107,338,54,270.0,40,300.0,36


### Remove combinations

In [17]:
def remove_combination(df_db, angle_init, weight_limit, df_comb, side):
    # db中的實際填補角度以最接近的欲填補角度替代
    for i in range(len(df_db)):
        closest_index = np.argmin(np.abs(angle_init - df_db.loc[i, f"{side.lower()}_angle_act"]))
        df_db.loc[i, f"{side.lower()}_angle_act"] = angle_init[closest_index]

    # 計算已填補角度的剩餘可填補重量
    cumulative_weight = df_db.groupby(f"{side.lower()}_angle_act").agg({f"{side.lower()}_weight_act": sum})
    cumulative_weight = cumulative_weight.to_dict()[f"{side.lower()}_weight_act"]

    # 刪除超出可填補重量的候選組合
    for key, value in cumulative_weight.items():
        if value > 0:
            remain_min = 0
            remain_max = weight_limit - value
        else:
            remain_min = (- weight_limit) - value
            remain_max = 0

        drop_index = df_comb.query(f"平衡_{side}側角度 == {key} and (平衡_{side}側配重 > {remain_min}) and (平衡_{side}側配重 < {remain_max})").index
        df_comb = df_comb.drop(drop_index).reset_index(drop = True)
    
    return df_comb

In [18]:
if not df_db.empty:
    df_db = df_db.fillna(0)
    df_comb = remove_combination(df_db, l_angle_init, weight_limit, df_comb, side = "L")
    df_comb = remove_combination(df_db, f_angle_init, weight_limit, df_comb, side = "F")

df_pred = df_comb.copy()
print(df_pred.shape)

(10206, 9)


### Generate features

In [19]:
df_pred["預期_F側不平衡量"] = df_pred["初始_F側不平衡量"] - df_pred["平衡_F側配重"]
df_pred["預期_L側不平衡量"] = df_pred["初始_L側不平衡量"] - df_pred["平衡_L側配重"]

df_pred["初始_不平衡量_diff"] = abs(df_pred["初始_L側不平衡量"] - df_pred["初始_F側不平衡量"])
df_pred["平衡_不平衡量_diff"] = abs(df_pred["平衡_L側配重"] - df_pred["平衡_F側配重"])

diff = abs(df_pred["初始_L側角度"] - df_pred["初始_F側角度"])
df_pred["初始_角度_diff"] = diff.apply(lambda X: min(X, 360 - X))
diff = abs(df_pred["平衡_L側角度"] - df_pred["平衡_F側角度"])
df_pred["平衡_角度_diff"] = diff.apply(lambda X: min(X, 360 - X))
df_pred.head(3)

,平衡轉速,初始_L側角度,初始_L側不平衡量,初始_F側角度,初始_F側不平衡量,平衡_L側角度,平衡_L側配重,平衡_F側角度,平衡_F側配重,預期_F側不平衡量,預期_L側不平衡量,初始_不平衡量_diff,平衡_不平衡量_diff,初始_角度_diff,平衡_角度_diff
0,1000,315,107,338,54,270.0,40,300.0,34,20,67,53,6,23,30.0
1,1000,315,107,338,54,270.0,40,300.0,35,19,67,53,5,23,30.0
2,1000,315,107,338,54,270.0,40,300.0,36,18,67,53,4,23,30.0


In [20]:
l_df, f_df = df_pred.copy(), df_pred.copy()

### Feature engineering

In [21]:
for col in features[1:]:    
    max_ = l_outlier_boundary[col]["max"]
    max_index = l_df.index[l_df[col] > max_].to_list()
    l_df.loc[max_index, col] = max_

    min_ = l_outlier_boundary[col]["min"]
    min_index = l_df.index[l_df[col] < min_].to_list()
    l_df.loc[min_index, col] = min_

    max_ = f_outlier_boundary[col]["max"]
    max_index = f_df.index[f_df[col] > max_].to_list()
    f_df.loc[max_index, col] = max_

    min_ = f_outlier_boundary[col]["min"]
    min_index = f_df.index[f_df[col] < min_].to_list()
    f_df.loc[min_index, col] = min_

In [22]:
l_df[l_skew_feat] = l_pt.transform(l_df[l_skew_feat])
f_df[f_skew_feat] = f_pt.transform(f_df[f_skew_feat])
l_df[features]    = l_scaler.transform(l_df[features])
f_df[features]    = f_scaler.transform(f_df[features])

### Predict

In [23]:
pred_l = l_model.predict(l_df)
pred_f = f_model.predict(f_df)

In [24]:
df_comb["最終_L側不平衡量"] = pred_l
df_comb["最終_F側不平衡量"] = pred_f

### Find best combination

In [25]:
df_comb["初始_總不平衡量"] = df_comb.eval("(abs(初始_L側不平衡量) + abs(初始_F側不平衡量))")
df_comb["最終_總不平衡量"] = df_comb.eval("(abs(最終_L側不平衡量) + abs(最終_F側不平衡量))")

# 正常不可能低於0.1
df_comb1 = df_comb.query("(最終_L側不平衡量 >= 0.1) and (最終_F側不平衡量 >= 0.1)")
if not df_comb1.empty:
    df_comb = df_comb1.reset_index(drop = True)

# 先嚴格篩選，再寬鬆篩選
df_comb2 = df_comb.query("((平衡_L側配重 > 0) and (平衡_F側配重 > 0)) and (最終_總不平衡量 < 初始_總不平衡量)")
if df_comb2.empty:
    df_comb2 = df_comb.query("(最終_總不平衡量 < 初始_總不平衡量)")
if df_comb2.empty:
    df_comb2 = df_comb.copy()

# 抓出最終_總不平衡量最小的作為回傳結果
comb_index1 = df_comb2["最終_總不平衡量"].idxmin()
result1 = df_comb.loc[[comb_index1]]
result1

,平衡轉速,初始_L側角度,初始_L側不平衡量,初始_F側角度,初始_F側不平衡量,平衡_L側角度,平衡_L側配重,平衡_F側角度,平衡_F側配重,最終_L側不平衡量,最終_F側不平衡量,初始_總不平衡量,最終_總不平衡量
5049,1000,315,107,338,54,330.0,60,330.0,34,12.993604,10.052241,161,23.045844


### Find second combination

In [26]:
l_weight_fill = result1.loc[comb_index1, "平衡_L側配重"]
f_weight_fill = result1.loc[comb_index1, "平衡_F側配重"]

# 平衡配重的個位數去尾後的值
l_quotient  = (l_weight_fill // 10) * 10
f_quotient  = (f_weight_fill // 10) * 10

# 產生可能的次要可行解組合，ex: [n, n+5, n+10]
l_answers = [l_quotient, l_quotient + 5, l_quotient + 10] if (l_quotient >= 30) else []
f_answers = [f_quotient, f_quotient + 5, f_quotient + 10] if (f_quotient >= 30) else []

# 抓出符合的組合
if (l_quotient >= 30) and (f_quotient >= 30): # 兩個都大於30
    df_comb3 = df_comb2.query(f"(平衡_L側配重 in {l_answers}) and (平衡_F側配重 in {f_answers})")
elif (l_quotient >= 30) or (f_quotient >= 30): # 其中一個大於30 
    df_comb3 = df_comb2.query(f"((平衡_L側配重 in {l_answers}) and (平衡_F側配重 == {f_weight_fill})) or ((平衡_F側配重 in {f_answers}) and (平衡_L側配重 == {l_weight_fill}))")

# 抓出最終_總不平衡量最小的作為回傳結果
if ('df_comb3' in locals()) and (len(df_comb3) > 0):
    comb_index2 = df_comb3["最終_總不平衡量"].idxmin()
    result2 = df_comb.loc[[comb_index2]]
else:
    result2 = pd.DataFrame()
result2

,平衡轉速,初始_L側角度,初始_L側不平衡量,初始_F側角度,初始_F側不平衡量,平衡_L側角度,平衡_L側配重,平衡_F側角度,平衡_F側配重,最終_L側不平衡量,最終_F側不平衡量,初始_總不平衡量,最終_總不平衡量
5050,1000,315,107,338,54,330.0,60,330.0,35,13.017187,10.216918,161,23.234105


In [27]:
results = []
result_col = ["l_angle_pred", "l_weight_pred", "f_angle_pred", "f_weight_pred"]
result1 = result1[['平衡_L側角度', '平衡_L側配重', '平衡_F側角度', '平衡_F側配重']].astype(str)
result1.columns = result_col
result1 = result1.to_dict(orient = "records")[0]
results.append(result1)

if len(result2) > 0:
    result2 = result2[['平衡_L側角度', '平衡_L側配重', '平衡_F側角度', '平衡_F側配重']].astype(str)
    result2.columns = result_col
    result2 = result2.to_dict(orient = "records")[0]
    results.append(result2)
    
print(f"results = {results}")

results = [{'l_angle_pred': '330.0', 'l_weight_pred': '60', 'f_angle_pred': '330.0', 'f_weight_pred': '34'}, {'l_angle_pred': '330.0', 'l_weight_pred': '60', 'f_angle_pred': '330.0', 'f_weight_pred': '35'}]


### Save output

In [28]:
result = {
    "status":   "success",
    "reason":   "",
    "work_id":  work_id,
    "op":       op,
    "model_id": model_id,
    "predict":  results
}
result

{'status': 'success',
 'reason': '',
 'work_id': 'FC239512',
 'op': '1',
 'model_id': '20230807120000000',
 'predict': [{'l_angle_pred': '330.0',
   'l_weight_pred': '60',
   'f_angle_pred': '330.0',
   'f_weight_pred': '34'},
  {'l_angle_pred': '330.0',
   'l_weight_pred': '60',
   'f_angle_pred': '330.0',
   'f_weight_pred': '35'}]}

In [29]:
with open(output_json, 'w') as file:
    json.dump(result, file, indent = 4)